In [1]:
import numpy as np
import pygalmesh
import SimpleITK as sitk
from nanomesh.utils import show_slice, show_volume, generate_mesh_from_binary_image
from nanomesh.periodic_utils import insert_periodic_info

class Cross(pygalmesh.DomainBase):
    def __init__(self):
        super().__init__()
        
        self.npts = 50
        self.size = 1
        self.x = np.linspace(0,self.size,self.npts)
        self.data = np.zeros((self.npts,self.npts,self.npts))
        
        
        idx = (self.x>0.25*self.size) & (self.x<0.75*self.size)
        all_idx = list(range(self.npts))
        self.data[np.ix_(idx,idx,all_idx)] = 1.
        self.data[np.ix_(idx,all_idx,idx)] = 1.
        self.data[np.ix_(all_idx,idx,idx)] = 1.
                    

    def eval(self, x):
        ix = np.argmin(np.abs(x[0]-self.x))
        iy = np.argmin(np.abs(x[1]-self.x))
        iz = np.argmin(np.abs(x[2]-self.x))
        if self.data[ix,iy,iz] == 1:
            return -1.
        else:
            return 0.


In [3]:
mesh = pygalmesh.generate_periodic_mesh(
    Cross(),
    [0, 0, 0, 1, 1, 1],
    max_cell_circumradius=0.05,
    min_facet_angle=30,
    max_radius_surface_delaunay_ball=0.05,
    max_facet_distance=0.025,
    max_circumradius_edge_ratio=2.0,
    number_of_copies_in_output=4,
    # odt=True,
    # lloyd=True,
    verbose=False,
)

import pyvista as pv
pv.plot_itk(mesh)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [3]:
mesh = insert_periodic_info(mesh, [0, 0, 0, 1, 1, 1])

In [7]:
import meshio
meshio.write('cross_periodic.msh', mesh, file_format='gmsh22', binary=False)